In [4]:
import pandas as pd



df = pd.read_csv(r'/Users/kellan/Downloads/ufo-reports-master/csv-data/ufo-scrubbed-geocoded-time-standardized.csv')

df.head()

,Date,City,State,Country,Type,Seconds,Duration,Description,Date added,Lat,Lon
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [7]:
ufos_canada = df[df['Country'] == 'ca']

ufos_canada.to_csv('ufos_canada.csv')